# Multiple Object Detection, Tracking and Reidentification

This notebook documents an experimental computer vision pipeline for object detection, tracking, and re-identification.
Architectures used
- Object Detection: YOLOv8
- Object Tracking: BoTSORT
- Object Reidentification: Pretreined weight Osnet_x_1.0

The focus is on:
- Comparing multiple state-of-the-art trackers
- Analyzing qualitative tracking and re-identification behavior
- Understanding failure cases under occlusion, camera motion, and viewpoint changes

This is an experimental / research notebook rather than a production-ready pipeline.
*These code was adapted from https://github.com/mikel-brostrom/yolov8_tracking and works on python>3.7*


## 1 Setting the Path to frames

In [1]:
import os

In [2]:
try:
    NOTEBOOK_DIR = os.path.dirname(os.path.abspath("__file__"))
except:
    NOTEBOOK_DIR = os.getcwd()

PROJECT_ROOT = os.path.abspath(os.path.join(NOTEBOOK_DIR, ".."))
VIDEO_PATH = os.path.join(PROJECT_ROOT, "data", "cups_video.mov")

print("Video path:", VIDEO_PATH)
print("Video exists:", os.path.exists(VIDEO_PATH))

Video path: /Users/tomiwa2/Downloads/Courses/H23/INF6804/TP/TP3/Submission 3/data/cups_video.mov
Video exists: True


## 2. Downloading amd installing the files and tools needed for the experiment

In [3]:
%cd {PROJECT_ROOT}
%pip install -r requirements.txt
%pip install -e .

/Users/tomiwa2/Downloads/Courses/H23/INF6804/TP/TP3/Submission 3
Note: you may need to restart the kernel to use updated packages.
Obtaining file:///Users/tomiwa2/Downloads/Courses/H23/INF6804/TP/TP3/Submission%203
ERROR: file:///Users/tomiwa2/Downloads/Courses/H23/INF6804/TP/TP3/Submission%203 does not appear to be a Python project: neither 'setup.py' nor 'pyproject.toml' found.
Note: you may need to restart the kernel to use updated packages.


In [4]:
%cd {PROJECT_ROOT}
!git clone --recurse-submodules https://github.com/mikel-brostrom/yolov8_tracking.git
%cd yolov8_tracking

/Users/tomiwa2/Downloads/Courses/H23/INF6804/TP/TP3/Submission 3
fatal : le chemin de destination 'yolov8_tracking' existe déjà et n'est pas un répertoire vide.
/Users/tomiwa2/Downloads/Courses/H23/INF6804/TP/TP3/Submission 3/yolov8_tracking


In [5]:
MODEL = "yolov8m.pt"
from ultralytics import YOLO

model = YOLO(MODEL)

model.fuse()

YOLOv8m summary (fused): 92 layers, 25,886,080 parameters, 0 gradients, 78.9 GFLOPs


In [6]:
# Checking the id for the cup class from the model
CLASS_NAMES_DICT = model.model.names
CUP_ID = [k for k,v in CLASS_NAMES_DICT.items() if v == 'cup'][0]
CUP_ID

41

## 3. Performing the experiments
The following architectures were used to find the best one for the task.

### 3.1 Detectors
- yolov8m.pt

### 3.2 Trackers
- BoT-SORT
- ByteTrack
- OCSort
- StrongSORT
- DeepOCSSort

### 3.3 Reidentification (I experimented with best performing Reidentification weights for Same-domain reidentification) 
https://kaiyangzhou.github.io/deep-person-reid/MODEL_ZOO
- osnet_x0_25
- osnet_x0_5
- osnet_x0_75
- osnet_x1_0

*Results for each combination can be found here*
- Tracking Result for BoT-SORT: https://drive.google.com/file/d/1ig468Lotq1-9C0KcwxMXdimtjd4uME6K/view?usp=sharing
- Tracking Result for Other Trackers: https://drive.google.com/file/d/15WbumWJHVCj0pYhdbZWRRZQv44i6TTBS/view?usp=sharing

### Object Tracking

In [7]:
%cd {PROJECT_ROOT}

model.track(
    source=VIDEO_PATH,
    tracker="botsort.yaml",  
    classes=[CUP_ID],
    save=True,
    line_width=3,
    save_txt=True,
    project="results",
    name="botsort"
)

/Users/tomiwa2/Downloads/Courses/H23/INF6804/TP/TP3/Submission 3

WARNING ⚠️ 
Inference results will accumulate in RAM unless `stream=True` is passed, which can cause out-of-memory errors for large
sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/1385) /Users/tomiwa2/Downloads/Courses/H23/INF6804/TP/TP3/Submission 3/data/cups_video.mov: 384x640 1 cup, 138.0ms
video 1/1 (frame 2/1385) /Users/tomiwa2/Downloads/Courses/H23/INF6804/TP/TP3/Submission 3/data/cups_video.mov: 384x640 1 cup, 123.3ms
video 1/1 (frame 3/1385) /Users/tomiwa2/Downloads/Courses/H23/INF6804/TP/TP3/Submission 3/data/cups_video.mov: 384x640 1 cup,

[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange', 50: 'broccoli', 51: 'carrot', 52: 'hot dog', 53: 'pizza', 54: 'donut', 55: 'cake', 56: 'chair', 57: 'couch', 58: 'potted p

### Model Evaluation on MOT20

In [8]:
!git clone https://github.com/JonathonLuiten/TrackEval.git
%pip install -r TrackEval/requirements.txt

Clonage dans 'TrackEval'...
remote: Enumerating objects: 924, done.
remote: Counting objects: 100% (378/378), done.
remote: Compressing objects: 100% (59/59), done.
remote: Total 924 (delta 344), reused 319 (delta 319), pack-reused 546 (from 1)
Réception d'objets: 100% (924/924), 393.80 Kio | 7.29 Mio/s, fait.
Résolution des deltas: 100% (619/619), fait.
  Using cached numpy-1.18.1.zip (5.4 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... error
  error: subprocess-exited-with-error
  
  × Preparing metadata (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [28 lines of output]
      Running from numpy source directory.
      <string>:461: UserWarning: Unrecognized setuptools command, proceeding with generating Cython sources and expanding templates
      Traceback (most recent call last):
        File "/opt/anaconda3/lib/python3.13/site-packages/pip/_vendor/pyproject_hooks/_in_process/_

In [23]:
'''
This handles the model evaluation on the MOT20 data
'''
!python val.py --yolo-weights yolov8m.pt --benchmark MOT20 --tracking-method botsort --reid-weights osnet_x0_25_market1501.pt

python: can't open file '/Users/tomiwa2/Downloads/Courses/H23/INF6804/TP/TP3/Submission 3/yolov8_tracking/val.py': [Errno 2] No such file or directory
